In [140]:
import json
import pennylane as qml
import pennylane.numpy as np

In [141]:
def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """
    #hamiltonian=0
    coefs=[]
    obs=[]
    for i in range(4):
        obs.append(qml.PauliX(i))
        coefs.append(-h)
        
    for i in range(4):
        obs.append(qml.PauliZ(i)@qml.PauliZ((i+1)%4))
        coefs.append(-1)
        
    hamiltonian=qml.Hamiltonian(coefs, obs)
        
    return hamiltonian

In [147]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """
    layers=4

    for i in range(layers):
        for j in range(4):
            qml.RX(params[i*2*4+2*j],wires=j)
            qml.RZ(params[i*2*4+2*j+1],wires=j)
        for j in range(4):
            qml.CNOT(wires=[j,(j+1)%4])
    for j in range(4):
        qml.RX(params[4*2*layers+2*j],wires=j)
        qml.RZ(params[4*2*layers+2*j],wires=j)
        
    return qml.expval(H)

In [145]:
def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """
    params = np.random.normal(0, np.pi, (4+1)*4*2, requires_grad=True)
    opt = qml.AdamOptimizer(0.5)#qml.AdamOptimizer(0.3)#qml.optimize.NesterovMomentumOptimizer(0.5)#qml.AdamOptimizer(0.5)
    H=create_Hamiltonian(h)
    for j in range(300):
        params,_ = opt.step(model, params, H)
    return params
    

In [149]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."


In [150]:
test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038']]

In [151]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
Correct!
Running test case 1 with input '2.3'...
Correct!


In [148]:
h=2.3
params=train(h)
H=create_Hamiltonian(h)
model(params, H)

tensor(-9.63869428, requires_grad=True)

In [67]:
create_Hamiltonian(h)

  (-1.0) [X0]
+ (-1.0) [X1]
+ (-1.0) [X2]
+ (-1.0) [X3]
+ (-1) [Z0 Z1]
+ (-1) [Z1 Z2]
+ (-1) [Z2 Z3]
+ (-1) [Z3 Z0]


In [70]:
h=1.0
weights = np.random.normal(0, np.pi, 2*2*4, requires_grad=True)
H=create_Hamiltonian(h)
model(weights, H)

params' len: 16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


tensor(0.9177024, requires_grad=True)